In [1]:
import itertools
import os
import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import seaborn as sns

pp = pprint.PrettyPrinter(indent=4)

print("Using pandas %s version" % pd.__version__)
print("Using seaborn %s version" % sns.__version__)
print("Using scipy %s version" % scipy.__version__)

Using pandas 1.1.3 version
Using seaborn 0.11.0 version
Using scipy 1.5.2 version


In [2]:
splits = [
    "RandomKFold",
    "GroupShuffleSplit",
    "ShuffleSplit",
    "StratifiedShuffleSplit",
    "StratifiedKFold",
    "KFold",
    "TimeSeriesSplit",
]

splitsWithoutGroupShuffleSplit = list(
    filter(lambda x: (x != "GroupShuffleSplit"), splits)
)

data_dir = r"C:\Projects\RecSys2020\results\new2\\"
#data_dir = r"E:\OneDrive\MSThesis\_2019\master_thesis_template_vu_mif_cs1-master\master_thesis_template_vu_mif_cs1-master\_third_part\img\\"

imfFull = "#548235"
imfTest = "#99C979"

knnFullTest = "#C55A11"
normalizedKnnFull = "#4A76C6"
normalizedKnnTest = "#19C3FF"

average = "#FFC000"
popularity = "#F18F8F"
random = "#BFBFBF"

# colors = ('red','blue','green','magenta', 'black', 'purple', 'grey', 'orange')
rec_colors = (
    imfFull,
    imfTest,
    knnFullTest,
    normalizedKnnFull,
    normalizedKnnTest,
    average,
    popularity,
    random,
)


def split_list(a_list):
    half = len(a_list) // 2
    return a_list[:half], a_list[half:]

In [3]:
def plot_figure1(split, ax1, ax2):
    print(split)
    # ax1 = ax[i,j]
    ax1.set_title(split)

    # ax2 = ax[i,j+1]
    ax2.set_title(split)

    fig1 = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure1.txt",
        sep="\t",
        header=4,
    )

    max = fig1.max()[1:].max()
    if max < 0.005:
        print(max)
        fig1["Full"] = fig1["Full"].multiply(1000)
        fig1["Test"] = fig1["Test"].multiply(1000)

    f1 = np.array(fig1)
    df = pd.DataFrame(index=["Full", "Test"])

    for i, r in enumerate(fig1.Recommender):
        df[r] = f1[i][1:]

    ax00 = df.plot(kind="bar", color=rec_colors, legend=None, ax=ax1)
    # ,figsize=(8,12)
    ax00.set_ylabel("Precision@10")
    ax00.xaxis.set_tick_params(rotation=0)
    # plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Recommender', title_fontsize='xx-large')

    data = [
        list(a)
        for a in zip(
            fig1.Recommender,
            fig1.Full.rank(ascending=False),
            fig1.Test.rank(ascending=False),
        )
    ]

    artists = []
    for row, color in zip(data, rec_colors):
        artist = plt.Line2D(
            xdata=[1, 2], ydata=[row[1:]], lw=1, color=color, marker="o"
        )
        ax2.add_artist(artist)
        artists.append(artist)

    ax2.set_ybound([0.5, 8.3])
    ax2.set_xbound([0.94, 2.06])
    ax2.spines["top"].set_visible(False)
    ax2.spines["right"].set_visible(False)
    ax2.spines["bottom"].set_visible(False)
    ax2.spines["left"].set_visible(False)
    ax2.invert_yaxis()
    ax2.set_xticks([1, 2])
    ax2.set_xticklabels(["Full", "Test"])

    ax2.set_ylabel("Sistemos reitingas")

    return (artists, fig1.Recommender)

In [21]:
print("figure1.cross-validation.png\n")
fig, ax = plt.subplots(len(splitsWithoutGroupShuffleSplit) // 2, 2 * 2)
firstColumn, secondColumn = split_list(splitsWithoutGroupShuffleSplit)

for i, split in enumerate(firstColumn):
    ax1 = ax[i, 0]
    ax2 = ax[i, 1]
    plot_figure1(split, ax1, ax2)

for i, split in enumerate(secondColumn):
    ax1 = ax[i, 2]
    ax2 = ax[i, 3]
    artists, recommenders = plot_figure1(split, ax1, ax2)

for idx, a in np.ndenumerate(ax):
    a.tick_params(axis="both", which="both", pad=0)

plt.legend(
    artists,
    recommenders,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="xx-large",
)

fig.set_size_inches(10, 9)
fig.tight_layout()

png = os.path.join(data_dir, "figure1.cross-validation.png")
fig.savefig(png, format="png", dpi=300)
plt.show()
plt.close("all")

figure1.cross-validation.png

RandomKFold
0.0006556178421054435
ShuffleSplit
StratifiedShuffleSplit
StratifiedKFold
0.0005575733073188304
KFold
0.00030455291552809814
TimeSeriesSplit
0.0006556178421054435


In [25]:
print("figure1.latest.png\n")
fig, ax = plt.subplots(1, 2)
firstColumn, secondColumn = split_list(splitsWithoutGroupShuffleSplit)


split = 'ShuffleSplit'
ax1 = ax[0]
ax2 = ax[1]
artists, recommenders = plot_figure1(split, ax1, ax2)

for idx, a in np.ndenumerate(ax):
    a.tick_params(axis="both", which="both", pad=0)

plt.legend(
    artists,
    recommenders,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="xx-large",
)

fig.set_size_inches(8, 6)
fig.tight_layout()

png = os.path.join(data_dir, "figure1.latest.png")
fig.savefig(png, format="png", dpi=300)
plt.show()
plt.close("all")

figure1.latest.png

ShuffleSplit


In [5]:
print("figure1.cross-validation.b.png\n")
fig1splits = list(filter(lambda x: (x != "GroupShuffleSplit"), splits))
# fig, ax = plt.subplots(int(len(fig1splits)/2), 2 * 2)
def split_list(a_list):
    half = len(a_list) // 2
    return a_list[:half], a_list[half:]


firstColumn, secondColumn = split_list(fig1splits)
fig = plt.figure(figsize=(8, 6))
# ax0.plot(x, y)
# ax1.plot(y, x)

# plt.tight_layout()
# plt.show()
shape = (3, 9)
for i, split in enumerate(firstColumn):
    # print(i)
    ax1 = plt.subplot2grid(shape, (i, 0), colspan=2)
    ax2 = plt.subplot2grid(shape, (i, 2), colspan=1)
    plot_figure1(split, ax1, ax2)

for i, split in enumerate(secondColumn):
    # print(i)
    ax1 = plt.subplot2grid(shape, (i, 4), colspan=2)
    ax2 = plt.subplot2grid(shape, (i, 6))
    artists, recommenders = plot_figure1(split, ax1, ax2)

fig.set_size_inches(20, 15)
plt.legend(
    artists,
    recommenders,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="xx-large",
)
fig.tight_layout()

png = os.path.join(data_dir, "figure1.cross-validation.b.png")
# fig.savefig(png, format="png", dpi=300)
# plt.show()
plt.close("all")

figure1.cross-validation.b.png

RandomKFold
0.0006556178421054435
ShuffleSplit
StratifiedShuffleSplit
StratifiedKFold
0.0005575733073188304
KFold
0.00030455291552809814
TimeSeriesSplit
0.0006556178421054435


In [6]:
def rank(data):
    artists = []
    data.set_index("Target size")
    for i, j in data.groupby(["Target size"], axis=0):
        del j["Target size"]
        rank = j.rank(axis=1, ascending=False)
        artist = np.array(rank)[0]
        artists.append(artist)

    rank = []
    for i, t in enumerate(data.keys()[1:]):
        rank.append([t] + np.array(artists)[:, i].tolist())
    return rank

In [7]:
def plot_system_rankings(data, ax, xlabel=r"$|N_{u}|$", ylabel="", xticks=None):
    artists = []
    for row, color in zip(data, rec_colors):
        x = list(range(len(data[0]) - 1))
        artist = plt.Line2D(xdata=x, ydata=[row[1:]], lw=1, color=color, marker="o")
        ax.add_artist(artist)
        artists.append(artist)

    ax.set_ybound([0.8, 9])
    ax.set_xbound([-0.2, len(data) + 0.06])

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.set_xticks(list(range(len(data[0]) - 1)))
    ax.set_xticklabels(list(itertools.chain(xticks[:-1], ["Full"])))
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)

    # plt.xticks(list(range(len(ndcg_rank[0])-1)), ndcg['Target size'].array)
    ax.set_yticks(list(range(1, 9)))
    ax.invert_yaxis()
    ax.invert_xaxis()

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    return artists

In [8]:
print("plot_metrics")


def plot_metrics(
    data, ax, xlabel=r"$|N_{u}|$", ylabel="", twinx=None, title=None, colors=None
):
    if colors is None:
        colors = rec_colors

    artists = []
    for column_title, color in zip(data.columns[1:], colors):
        x = list(range(len(data[column_title])))
        artist = plt.Line2D(
            xdata=x, ydata=[data[column_title]], lw=1, color=color, marker="o"
        )
        ax.add_artist(artist)
        artists.append(artist)

    max = np.array((data.max()[1:])).max()
    if max < 0.0006:
        print('\t' + str(max))
        for c in data.columns[1:]:
            data[c] = data[c].multiply(1000)
            print('\t\t' + c)
        max = np.array((data.max()[1:])).max()

        #print(data)
        #fig1["Full"] = fig1["Full"].multiply(1000)
        #fig1["Test"] = fig1["Test"].multiply(1000)
        
    ax.set_ybound([0, max + max * 0.1 + 0.0000001])
    ax.set_xbound([-0.1, len(ndcg) + 0.06])

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.set_xticks(list(range(len(data["Target size"]))))
    ax.set_xticklabels(list(itertools.chain(data["Target size"].array[:-1], ["Full"])))
    ax.invert_xaxis()
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    if twinx is not None and twinx.max() > 0:
        #print("twinx", twinx)
        label = "Sum of p-values"
        ax2 = ax.twinx()
        (e,) = ax2.plot(
            list(range(len(twinx))),
            twinx,
            label=label,
            lw=1,
            color="black",
            fillstyle="none",
            markeredgecolor="black",
            marker="s",
        )
        ax2.set_ylabel(label)
        max = twinx.max()
        ax2.set_ylim(0, max + 0.1 * max + 0.01)

        ax2.spines["top"].set_visible(False)
        ax2.spines["right"].set_visible(False)
        ax2.spines["bottom"].set_visible(False)
        ax2.spines["left"].set_visible(False)
        # ax2.legend(loc=1)
        # artists.append(ax2)
        artists.append(e)

    if title is not None:
        ax.set_title(title)

    if xlabel != "":
        ax.set_xlabel(xlabel)
    return artists, data.columns[1:]

plot_metrics


In [9]:
print("figure3\n")
for split in splitsWithoutGroupShuffleSplit:
    print(split)
    ndcg = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
        sep="\t",
        header=4,
        nrows=13,
    )
    precision = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
        sep="\t",
        header=19,
        nrows=13,
    )
    recall = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
        sep="\t",
        header=34,
        nrows=13,
    )

    ndcg_rank = rank(ndcg)
    precision_rank = rank(precision)
    recall_rank = rank(recall)

    cols = ["Column {}".format(col) for col in range(1, 2)]
    rows = ["Row {}".format(row) for row in ["Precision@10", "Recall@10", "nDCG@10"]]

    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 8))

    # for ax, col in zip(axes[0], cols):
    #    ax.set_title(col)
    #    #ax.set_xlabel('|Nu|')

    for ax, row in zip(axes[:, 0], rows):
        ax.set_ylabel(row, rotation=90, size="large")

    plot_metrics(precision, axes[0][0], ylabel="Precision@10")
    plot_metrics(recall, axes[1][0], ylabel="Recall@10")
    artists, recommender = plot_metrics(ndcg, axes[2][0], ylabel="nDCG@10")

    plot_system_rankings(precision_rank, axes[0][1], xticks=ndcg["Target size"].array)
    plot_system_rankings(recall_rank, axes[1][1], xticks=ndcg["Target size"].array)
    plot_system_rankings(ndcg_rank, axes[2][1], xticks=ndcg["Target size"].array)

    plt.legend(
        artists,
        recommender,
        bbox_to_anchor=(1.05, 1),
        loc="upper left",
        # title="Recommender",
        title_fontsize="x-large",
    )
    fig.suptitle(split)

    fig.tight_layout()
    png = os.path.join(data_dir, "figure3." + split + ".png")
    fig.savefig(png, format="png", dpi=300)
    # dpi=1200
#plt.show()
plt.close("all")

figure3

RandomKFold
ShuffleSplit
StratifiedShuffleSplit
StratifiedKFold
	0.0005696396100880264
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
	0.00013694263431826299
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
KFold
	0.00030455291552809814
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
	8.008179772039368e-05
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
	0.000324500274554278
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
TimeSeriesSplit
	0.00024928987700699007
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
	

In [28]:
print("figure3.full.png\n")
split = "StratifiedShuffleSplit"
ndcg = pd.read_table(
    r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
    sep="\t",
    header=4,
    nrows=13,
)
precision = pd.read_table(
    r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
    sep="\t",
    header=19,
    nrows=13,
)
recall = pd.read_table(
    r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
    sep="\t",
    header=34,
    nrows=13,
)

ndcg_rank = rank(ndcg)
precision_rank = rank(precision)
recall_rank = rank(recall)

cols = ["Column {}".format(col) for col in range(1, 4)]
rows = [row for row in ["Precision@10", "Recall@10", "nDCG@10"]]

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(14, 8))

# for ax, col in zip(axes[0], cols):
#    ax.set_title(col)
#    #ax.set_xlabel('|Nu|')

for ax, row in zip(axes[:, 0], rows):
    ax.set_ylabel(row, rotation=90, size="large")

plot_metrics(precision, axes[0][0], ylabel="Precision@10")
plot_metrics(recall, axes[1][0], ylabel="Recall@10")
artists, recommender = plot_metrics(ndcg, axes[2][0], ylabel="nDCG@10")

plot_system_rankings(precision_rank, axes[0][1], xticks=ndcg["Target size"].array)
plot_system_rankings(recall_rank, axes[1][1], xticks=ndcg["Target size"].array)
plot_system_rankings(ndcg_rank, axes[2][1], xticks=ndcg["Target size"].array)

axes[0][0].set_title("MovieLens 1M")
axes[0][2].set_title("Yahoo R3")

## YAHOOO
ndcg = pd.read_table(
    r"C:\Projects\RecSys2020\results\figure3.txt",
    sep="\t",
    header=52,
    nrows=11,
)
precision = pd.read_table(
    r"C:\Projects\RecSys2020\results\figure3.txt",
    sep="\t",
    header=52,
    nrows=11,
)
recall = pd.read_table(
    r"C:\Projects\RecSys2020\results\figure3.txt",
    sep="\t",
    header=78,
    nrows=11,
)

ndcg_rank = rank(ndcg)
precision_rank = rank(precision)
recall_rank = rank(recall)

for ax, row in zip(axes[:, 0], rows):
    ax.set_ylabel(row, rotation=90, size="large")

plot_metrics(precision, axes[0][2], ylabel="Precision@10")
plot_metrics(recall, axes[1][2], ylabel="Recall@10")
artists, recommender = plot_metrics(ndcg, axes[2][2], ylabel="nDCG@10")

plot_system_rankings(precision_rank, axes[0][3], xticks=ndcg["Target size"].array)
plot_system_rankings(recall_rank, axes[1][3], xticks=ndcg["Target size"].array)
plot_system_rankings(ndcg_rank, axes[2][3], xticks=ndcg["Target size"].array)

## YAHOOO

plt.legend(
    artists,
    recommender,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="x-large",
)
# fig.suptitle(split)
fig.set_size_inches(13, 10)
fig.tight_layout()
png = os.path.join(data_dir, "figure3.full.png")
fig.savefig(png, format="png", dpi=300)
# dpi=1200
plt.show()
plt.close("all")

figure3.full.png



In [11]:
print("figure3.crossvalidation\n")


def plot_figure3(split, axes, row, col):

    ndcg = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
        sep="\t",
        header=4,
        nrows=13,
    )
    precision = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
        sep="\t",
        header=19,
        nrows=13,
    )
    recall = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure3.txt",
        sep="\t",
        header=34,
        nrows=13,
    )

    ndcg_rank = rank(ndcg)
    precision_rank = rank(precision)
    recall_rank = rank(recall)

    axes[0 + row][0 + col].set_title(
        split,
        fontdict={"size": 12, "weight": "bold"},
    )
    plot_metrics(precision, axes[0 + row][0 + col], ylabel="Precision@10", xlabel="")
    plot_metrics(recall, axes[1 + row][0 + col], ylabel="Recall@10", xlabel="")
    artists, recommender = plot_metrics(ndcg, axes[2 + row][0 + col], ylabel="nDCG@10")

    plot_system_rankings(
        precision_rank,
        axes[0 + row][1 + col],
        xlabel="",
        xticks=ndcg["Target size"].array,
    )
    plot_system_rankings(
        recall_rank, axes[1 + row][1 + col], xlabel="", xticks=ndcg["Target size"].array
    )
    plot_system_rankings(
        ndcg_rank, axes[2 + row][1 + col], xticks=ndcg["Target size"].array
    )
    return artists, recommender


fig, ax = plt.subplots(nrows=9, ncols=4)

firstColumn, secondColumn = split_list(splitsWithoutGroupShuffleSplit)


def plot_column(row, split, ax, col):
    print(split, row * 3)
    row = row * 3
    col = col * 2
    artists, recommenders = plot_figure3(split, ax, row, col)
    ax1.set_title(
        split, fontdict={"size": 12, "weight": "bold", "horizontalalignment": "right"}
    )
    return artists, recommenders


for i, split in enumerate(firstColumn):
    plot_column(i, split, ax, col=0)


for i, split in enumerate(secondColumn):
    artists, recommenders = plot_column(i, split, ax, col=1)


plt.legend(
    artists,
    recommender,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="x-large",
)
# fig.suptitle(split)

fig.set_size_inches(30, 25)
fig.tight_layout()
png = os.path.join(data_dir, "figure3.crossvalidation.png")
fig.savefig(png, format="png", dpi=300)
# dpi=1200
# plt.show()
plt.close("all")

figure3.crossvalidation

RandomKFold 0
ShuffleSplit 3
StratifiedShuffleSplit 6
StratifiedKFold 0
	0.0005696396100880264
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
	0.00013694263431826299
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
KFold 3
	0.00030455291552809814
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
	8.008179772039368e-05
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
	0.000324500274554278
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized kNN (test)
		Average Rating
		Popularity
		Random
TimeSeriesSplit 6
	0.00024928987700699007
		iMF (full)
		iMF (test)
		kNN (full/test)
		Normalized kNN (full)
		Normalized k

In [12]:
def fill_correlation(p, r, n):
    p = p.drop(index=[10, 11])
    r = r.drop(index=[10, 11])
    n = n.drop(index=[10, 11])

    CorrelationnDCG = [
        0.642857143,
        0.714285714,
        0.785714286,
        0.857142857,
        0.857142857,
        0.857142857,
        0.928571429,
        0.857142857,
        0.857142857,
        0.785714286,
        0.714285714,
    ]
    CorrelationPreciion = [
        0.642857143,
        0.642857143,
        0.642857143,
        0.714285714,
        0.714285714,
        0.785714286,
        0.857142857,
        0.928571429,
        0.785714286,
        0.785714286,
        0.714285714,
    ]
    CorrelationRecall = [
        0.642857143,
        0.642857143,
        0.642857143,
        0.642857143,
        0.714285714,
        0.714285714,
        0.857142857,
        0.928571429,
        0.785714286,
        0.857142857,
        0.785714286,
    ]
    p["Correlation with unbiased evaluatio"] = CorrelationPreciion
    r["Correlation with unbiased evaluatio"] = CorrelationRecall
    n["Correlation with unbiased evaluatio"] = CorrelationnDCG

    def reoder_columns(d):
        cols = d.columns.tolist()
        cols = [cols[0]] + cols[-1:] + cols[1:-1]
        d = d[cols]
        return d

    p = reoder_columns(p)
    r = reoder_columns(r)
    n = reoder_columns(n)
    return (p, r, n)

In [13]:
print("def plot_figure4(...\n")


def plot_figure4(split, ax1, ax2, ax3, legend=True, yahoo=None):
    ndcg = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure4.txt.ml1m.txt",
        sep="\t",
        header=4,
        nrows=13,
    )
    precision = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure4.txt.ml1m.txt",
        sep="\t",
        header=19,
        nrows=13,
    )
    recall = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure4.txt.ml1m.txt",
        sep="\t",
        header=34,
        nrows=13,
    )

    cols = ["Column {}".format(col) for col in range(1, 2)]
    rows = ["Row {}".format(row) for row in ["Precision@10", "Recall@10", "nDCG@10"]]

    for ax, row in zip([ax1, ax2, ax3], rows):
        ax.set_ylabel(row, rotation=90, size="large")

    twinx = "Sum of p-values"
    CorrelationWithUnbiasedEvaluation = "#C00000"
    ExpectedIntersectionRatioInTopN = "#FFC000"
    RatioOfTies = "#3333FF"
    RatioOfTiesAtZero = "#006600"
    fig4colors = (
        # CorrelationWithUnbiasedEvaluation,
        ExpectedIntersectionRatioInTopN,
        RatioOfTies,
        RatioOfTiesAtZero,
    )

    # precision, recall, ndcg = fill_correlation(precision, recall, ndcg)
    precision.head()
    plot_metrics(
        precision.iloc[:, :-1],
        ax1,
        ylabel="Precision@10",
        twinx=precision[twinx],
        colors=fig4colors,
        xlabel="",
    )
    plot_metrics(
        recall.iloc[:, :-1],
        ax2,
        ylabel="Recall@10",
        twinx=recall[twinx],
        colors=fig4colors,
        xlabel="",
    )
    artists, recommender = plot_metrics(
        ndcg.iloc[:, :-1],
        ax3,
        ylabel="nDCG@10",
        twinx=ndcg[twinx],
        colors=fig4colors,
    )

    if yahoo is not None:
        print("yahoo")
        ax1, ax2, ax3 = yahoo
        ndcg = pd.read_table(
            r"C:\Projects\RecSys2020\results\figure4.txt.yahoo.txt",
            sep="\t",
            header=4,
            nrows=11,
        )
        precision = pd.read_table(
            r"C:\Projects\RecSys2020\results\figure4.txt.yahoo.txt",
            sep="\t",
            header=17,
            nrows=11,
        )
        recall = pd.read_table(
            r"C:\Projects\RecSys2020\results\figure4.txt.yahoo.txt",
            sep="\t",
            header=30,
            nrows=11,
        )
        fig4colors = (
            CorrelationWithUnbiasedEvaluation,
            ExpectedIntersectionRatioInTopN,
            RatioOfTies,
            RatioOfTiesAtZero,
        )
        plot_metrics(
            precision.iloc[:, :-1],
            ax1,
            ylabel="Precision@10",
            twinx=precision[twinx],
            colors=fig4colors,
            xlabel="",
        )
        plot_metrics(
            recall.iloc[:, :-1],
            ax2,
            ylabel="Recall@10",
            twinx=recall[twinx],
            colors=fig4colors,
            xlabel="",
        )
        artists, recommender = plot_metrics(
            ndcg.iloc[:, :-1],
            ax3,
            ylabel="nDCG@10",
            twinx=ndcg[twinx],
            colors=fig4colors,
        )

    if legend == True:
        plt.legend(artists, recommender, bbox_to_anchor=(1.05, 1), loc="best")
        fig.suptitle(split)

    return artists, ndcg.columns[1:]

def plot_figure4(...



In [14]:
print("figure4.full\n")

split = "StratifiedShuffleSplit"

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 8))

artists, recommender = plot_figure4(
    split,
    axes[0, 0],
    axes[1, 0],
    axes[2, 0],
    legend=False,
    yahoo=(axes[0, 1], axes[1, 1], axes[2, 1]),
)

axes[0][0].set_title("MovieLens 1M")
axes[0][1].set_title("Yahoo R3")

plt.legend(
    artists,
    recommender,
    loc="upper center",
    bbox_to_anchor=(0.2, -0.05),
    fancybox=True,
    shadow=True,
    ncol=2,
)
# plt.legend(artists, recommender, bbox_to_anchor=(1.05, 1), loc="best")

box = plt.gca().get_position()
plt.gca().set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])

# Put a legend below current axis
#plt.legend(
#    loc="upper center", bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=5
#)


fig.set_size_inches(10, 6)
#fig.tight_layout()
plt.tight_layout()

    
png = os.path.join(data_dir, "figure4.full.png")
fig.savefig(png, format="png", dpi=300)
# dpi=1200
#plt.show()
plt.close("all")

figure4.full

yahoo


In [15]:
print("figure4\n")

for split in splitsWithoutGroupShuffleSplit:
    print(split)

    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 8))
    artists = plot_figure4(split, axes[0], axes[1], axes[2])
    png = os.path.join(data_dir, "figure4.txt.ml1m." + split + ".png")
    fig.set_size_inches(8, 6)
    fig.tight_layout()
    fig.savefig(png, format="png", dpi=300)
    # dpi=1200
# plt.show()
plt.close("all")

figure4

RandomKFold
ShuffleSplit
StratifiedShuffleSplit
StratifiedKFold
KFold
TimeSeriesSplit


In [16]:
print("figure4.cross-validation.png\n")
fig, ax = plt.subplots(9, len(splitsWithoutGroupShuffleSplit) // 3)
firstColumn, secondColumn = split_list(splitsWithoutGroupShuffleSplit)


def plot_column(i, split, ax, col):
    print(split, i * 3)
    i = i * 3
    ax1 = ax[0 + i, col]
    ax2 = ax[1 + i, col]
    ax3 = ax[2 + i, col]
    artists, recommenders = plot_figure4(split, ax1, ax2, ax3, legend=False)
    ax1.set_title(split, fontdict={"size": 12, "weight": "bold"})
    return artists, recommenders


for i, split in enumerate(firstColumn):
    plot_column(i, split, ax, col=0)


for i, split in enumerate(secondColumn):
    artists, recommenders = plot_column(i, split, ax, col=1)

plt.legend(
    artists,
    recommenders,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    title_fontsize="xx-large",
)
fig.set_size_inches(20, 20)
fig.tight_layout()

png = os.path.join(data_dir, "figure4.cross-validation.png")
fig.savefig(png, format="png", dpi=300)
# plt.show()
plt.close("all")

figure4.cross-validation.png

RandomKFold 0
ShuffleSplit 3
StratifiedShuffleSplit 6
StratifiedKFold 0
KFold 3
TimeSeriesSplit 6


In [17]:
print("figure5.cross-validation.png\n")
fig, axes = plt.subplots(nrows=len(splits), ncols=1, figsize=(8, 10))
for split, ax in zip(splits, axes):
    print(split)
    coverage = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure5.txt",
        sep="\t",
        header=4,
        nrows=13,
    )

    artists, recommenders = plot_metrics(
        coverage, ax, ylabel="Coverage@10", title=split
    )


plt.legend(
    artists,
    recommenders,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="x-large",
)

png = os.path.join(data_dir, "figure5.cross-validation.png")
# fig.suptitle(split)
fig.set_size_inches(10, 20)
fig.tight_layout()
fig.savefig(png, format="png", dpi=300)
# dpi=1200
# plt.show()
plt.close("all")

figure5.cross-validation.png

RandomKFold
GroupShuffleSplit
ShuffleSplit
StratifiedShuffleSplit
StratifiedKFold
KFold
TimeSeriesSplit


In [18]:
print("figure5.cross-validation.b.png\n")
fig, axs = plt.subplots(len(splitsWithoutGroupShuffleSplit) // 2, 2)
firstColumn, secondColumn = split_list(splitsWithoutGroupShuffleSplit)

for i, split in enumerate(firstColumn):
    coverage = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure5.txt",
        sep="\t",
        header=4,
        nrows=13,
    )
    print(split, i)
    ax = axs[i, 0]
    plot_metrics(coverage, ax, ylabel="Coverage@10", title=split)

for i, split in enumerate(secondColumn):
    coverage = pd.read_table(
        r"C:\Projects\RecSys2020\results\\" + split + r"\figure5.txt",
        sep="\t",
        header=4,
        nrows=13,
    )
    print(split, i)
    ax = axs[i, 1]
    artists, recommender = plot_metrics(coverage, ax, ylabel="Coverage@10", title=split)


plt.legend(
    artists,
    recommender,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="xx-large",
)
fig.set_size_inches(14, 8)
fig.tight_layout()
png = os.path.join(data_dir, "figure5.cross-validation.b.png")
fig.savefig(png, format="png", dpi=300)
# plt.show()
plt.close("all")

figure5.cross-validation.b.png

RandomKFold 0
ShuffleSplit 1
StratifiedShuffleSplit 2
StratifiedKFold 0
KFold 1
TimeSeriesSplit 2


In [19]:
print("figure5.full.png\n")
fig, (ax1, ax2) = plt.subplots(1, 2)
firstColumn, secondColumn = split_list(splitsWithoutGroupShuffleSplit)
split = "ShuffleSplit"
coverage = pd.read_table(
    r"C:\Projects\RecSys2020\results\\" + split + r"\figure5.txt",
    sep="\t",
    header=4,
    nrows=13,
)
plot_metrics(coverage, ax1, ylabel="Coverage@10", title="MovieLens1M")

coverage = pd.read_table(
    r"C:\Projects\RecSys2020\results\KFold\figure5.txt",
    sep="\t",
    header=4,
    nrows=11,
)
artists, recommender = plot_metrics(coverage, ax2, ylabel="Coverage@10", title="Yahoo!")


plt.legend(
    artists,
    recommender,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    # title="Recommender",
    title_fontsize="xx-large",
)
fig.set_size_inches(8, 3)
fig.tight_layout()
png = os.path.join(data_dir, "figure5.full.png")
fig.savefig(png, format="png", dpi=300)
#plt.show()
plt.close("all")

figure5.full.png



In [20]:
plt.close("all")
plt.show()